In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from scipy import spatial
from numpy import dot
from numpy.linalg import norm
from sklearn.metrics import mean_absolute_percentage_error
from tqdm import tqdm
import logging
import random
from datetime import datetime
import os
import seaborn as sns
import matplotlib.pyplot as plt
import multiprocessing
from math import sqrt
import time
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
import plotly.express as px
import re
from sklearn.metrics import mean_squared_error as mse

import psycopg2
from sklearn.model_selection import KFold
import plotly.figure_factory as ff
import plotly.graph_objects as go
import plotly.express as px
from tqdm import tqdm
import json

In [5]:
import warnings
warnings.filterwarnings("ignore")

In [6]:
try:
    connection = psycopg2.connect(user="RemoteFA",
                                    database="fa",
                                
                                    password="RFA_127_BT_fandc",
                                    host="rc1b-bdye1rzk75u6dgzf.mdb.yandexcloud.net",
                                    port="6432",
                                    sslmode = "allow")

except Exception as error:
    print("Ошибка при работе с PostgreSQL", error)


In [14]:
# хуйня
#####################################################################
########## ДАННЫЕ ######################
n_bundle = 102
type_data = 'train'
batch = 3 

def make_good_view(df, list_vac_skills):
    columns_to_drop = ['vacancies_type_id', 'is_shift', 'is_distance', 'is_multiple', 'description_hash2', 'is_junior', 'is_senior', 'part']
    if 'linked_bundles' in df.columns:
        columns_to_drop.append('linked_bundles')

    df.drop(columns_to_drop, axis=1, inplace=True)

    d_v3_region_bs = pd.read_sql('SELECT index as v3_region_index, new_region_id FROM public.d_v3_region_bs', connection)
    d_v3_new_region_id = pd.read_sql('SELECT new_id as new_region_id, region_name as region_name1,country_name FROM public.d_v3_new_region_id', connection)
    # добавляем про регион
    df = df.merge(d_v3_region_bs).merge(d_v3_new_region_id)
    df = df[df.country_name == 'Россия']
    df.insert(0, 'region_name', df.region_name1)
    df.insert(0, 'region', df.new_region_id)
    df = df.drop(['new_region_id', 'region_name1', 'country_name', 'region'], axis=1)
    # добавляем год
    df['first_pub_date'] = pd.to_datetime(df['first_pub_date'])
    df.insert(0, 'year', df['first_pub_date'].dt.year)

    df['new_salary'] = None # т.к. в дф в зп только наны
    df.rename(columns={'index': 'id'}, inplace=True)
    industry_df = pd.read_csv('./ind_groups_2.csv')
    df = df.merge(industry_df[['region_name', 'industry_group']], on='region_name', how='left')
    df = df[['id','salary_from_rub', 'source_site', 'year', 'new_salary', 'is_vahta', 'experience_id', 'region_name','industry_group','is_parttime']+list(list_vac_skills)]
    return df




def make_ans_df(based_path, n_bundle, type_data, batch):
    type_dict = {'train':1, 'test':2} # train/test
    batch_data = {1:'20230621', 2:'20230725', 3:'20230929'} # какая партия бундлов (по 25 шт)

    # базированные
    # based_path = './bundles_based_train_0929'
    based_list = [i for i in os.listdir(based_path) if 'based_b' in i]
    df_based = pd.read_csv(f'{based_path}/based_b_{n_bundle}.csv')

    # таблица где айди вакансии и соответствующий айдишник 
    df2 = pd.read_sql(f'SELECT * FROM public.v3_vac_type_id_{n_bundle}_part_{type_dict[type_data]}_bundles_{batch_data[batch]}', connection)
    df2.dropna(inplace=True)
    df2['bundle_id'] = df2['bundle_id'].astype('int')


    #####################################################################
    # сначала для всего сырого датафрейма из бд (пусть это будет df_start) проставляю навыки (1)
    df_start = pd.read_sql(f'SELECT * FROM public.v3_vacancies_all_type_id_{n_bundle}_part_{type_dict[type_data]}_{batch_data[batch]}', connection) # будущие финальные
    missing_ids = df_start.loc[~df_start['index'].isin(df_based['id']), 'index'].tolist() # айдишники которые есть в start_df (raw data), но нет в based
    start_df = df_start[df_start['index'].isin(missing_ids)]


    # названия столбцов с навыками из df_based
    list_vac_skills = df_based.columns[df_based.columns.tolist().index('is_parttime')+1:]
    for col_name in list_vac_skills:
        df_start[col_name] = None  


    # для данного бундла ключ (цифра айди навыка) - значение (полное название навыка)
    dict_temp = [re.search(r'\((\d+)_', i).group(1) for i in list_vac_skills]
    if len(set(dict_temp)) != len(list_vac_skills):
        print('ОПА БЕДЫ')
    dict_skills = {int(dict_temp[i]):list_vac_skills[i] for i in range(len(list_vac_skills))}


    # проставляем навыки
    # т.к. не все навыки из raw есть в based
    df2 = df2[df2['bundle_id'].isin(list(dict_skills.keys()))]
    df2 = df2[df2['v3_all_index'].isin(df_start['index'].values)]
    df_start.set_index('index', inplace=True)
    for index, row in df_start.iterrows():
        skills_one = df2[df2['v3_all_index']==index]['bundle_id'].values

        if skills_one != []:
            df_start.loc[index, [dict_skills[col] for col in skills_one]] = 1
    df_start.reset_index(inplace=True)

    # теперь оставляем только те строки в df_start, в которых есть навыки, но в зп нан
    ans_df = df_start[(~df_start[list_vac_skills].isna().all(axis=1)) & (df_start['salary_from_rub'].isna())]
    ans_df = make_good_view(ans_df, list_vac_skills)
    return ans_df



In [8]:
#bundles1 = [16, 27, 31, 33, 49, 52, 54, 58, 61, 69, 74, 91, 116, 119, 128, 134, 283, 336, 663, 706, 767, 783, 833, 836, 919]
bundles2 = [43, 23, 84, 85, 87, 124, 357, 561, 693, 712, 742, 765, 811, 812, 852, 853, 856, 873, 876, 888, 898, 917, 921, 954, 992]
# bundles3 = [2, 35, 40, 86, 102, 111, 320, 324, 325, 326, 337, 698, 720, 734, 760, 804, 829, 850, 904, 941, 982, 994, 996, 997, 998]

In [16]:

based_path = './bundles_based_test_0725' # папка где лежат based таблицы
type_data = 'test' 
batch = 2 # какая группа навыков? (1-ые 25, 2-ые, 3-и)

bundles2 = [23, 43, 84, 85, 87, 124, 357, 561, 693, 712, 742, 765, 811, 812, 852, 853, 856, 873, 876, 888, 898, 917, 921, 954, 992]

for n_bundle in bundles2:
    # постоянно вылетало, поэтому сюда добавила
    try:
        connection = psycopg2.connect(user="RemoteFA",
                                        database="fa",
                                    
                                        password="RFA_127_BT_fandc",
                                        host="rc1b-bdye1rzk75u6dgzf.mdb.yandexcloud.net",
                                        port="6432",
                                        sslmode = "allow")

    except Exception as error:
        print("Ошибка при работе с PostgreSQL", error)

    
    temp_df = make_ans_df(based_path, n_bundle, type_data, batch)
    temp_df.to_csv(f'./send/{batch}/{type_data}/{n_bundle}.csv', encoding='utf-8-sig', index = False)
    print(n_bundle, temp_df.shape)
    
    
    


23 (433, 73)
43 (14620, 171)
84 (715, 122)
85 (19091, 372)
87 (490, 134)
124 (31, 64)
357 (25, 57)
561 (2, 22)
693 (67, 31)
712 (121, 34)
742 (118, 25)
765 (968, 94)
811 (172, 48)
812 (262, 42)
852 (0, 20)
853 (76, 71)
856 (127, 67)
873 (33, 40)
876 (10, 19)
888 (1955, 76)
898 (1184, 49)
917 (1, 28)
921 (92, 50)
954 (206, 34)
992 (2099, 127)
